In [ ]:
import numpy as np
import librosa
import cv2
import moviepy.editor as mpy

def create_particles(num_particles, width, height):
    """
    画面上にランダムなパーティクルを生成する関数

    Args:
        num_particles (int): 生成するパーティクルの数
        width (int): 画面の幅
        height (int): 画面の高さ

    Returns:
        list: パーティクルのリスト。各パーティクルは [x座標, y座標, サイズ, 速度] のリストとして表される
    """
    particles = []
    for _ in range(num_particles):
        x = np.random.randint(0, width)
        y = np.random.randint(0, height)
        size = np.random.randint(2, 5)
        speed = np.random.randint(1, 5)
        particles.append([x, y, size, speed])
    return particles

def update_particles(particles, width, height, energy):
    """
    パーティクルの位置を更新する関数

    Args:
        particles (list): パーティクルのリスト
        width (int): 画面の幅
        height (int): 画面の高さ
        energy (float): 音声のエネルギー値 (0.0 - 1.0)

    Returns:
        list: 更新されたパーティクルのリスト
    """
    for p in particles:
        p[1] -= p[3] * energy  # パーティクルを下に移動させる
        if p[1] < 0:  # 画面上部からはみ出したパーティクルを処理
            p[1] = height
            p[0] = np.random.randint(0, width)
    return particles

def draw_particles(frame, particles, color):
    """
    パーティクルを描画する関数

    Args:
        frame (numpy.ndarray): 描画対象のフレーム
        particles (list): パーティクルのリスト
        color (tuple): パーティクルの色 (B, G, R)
    """
    for p in particles:
        cv2.circle(frame, (int(p[0]), int(p[1])), p[2], color, -1)  # パーティクルを描画

def create_mosaic_mask(height, width, num_blocks=30, min_block_size=20, max_block_size=150):
    """
    モザイク効果のためのマスクを生成する関数

    Args:
        height (int): マスクの高さ
        width (int): マスクの幅
        num_blocks (int, optional): モザイクブロックの数。デフォルトは30。
        min_block_size (int, optional): モザイクブロックの最小サイズ。デフォルトは20。
        max_block_size (int, optional): モザイクブロックの最大サイズ。デフォルトは150。

    Returns:
        numpy.ndarray: モザイクマスク
    """
    mask = np.ones((height, width), dtype=np.uint8) * 255  # 白いマスクを作成
    for _ in range(num_blocks):
        block_size = np.random.randint(min_block_size, max_block_size)
        x = np.random.randint(0, width - block_size)
        y = np.random.randint(0, height - block_size)
        cv2.rectangle(mask, (x, y), (x + block_size, y + block_size), 0, -1)  # ランダムな位置に黒い矩形を描画
    return mask

def apply_mosaic_effect(frame, mask, block_size=30):
    """
    フレームにモザイク効果を適用する関数

    Args:
        frame (numpy.ndarray): 処理対象のフレーム
        mask (numpy.ndarray): モザイクマスク
        block_size (int, optional): モザイクブロックのサイズ。デフォルトは30。

    Returns:
        numpy.ndarray: モザイク効果が適用されたフレーム
    """
    height, width = frame.shape[:2]
    small = cv2.resize(frame, (width // block_size, height // block_size))  # フレームを縮小
    mosaic = cv2.resize(small, (width, height), interpolation=cv2.INTER_NEAREST)  # 縮小したフレームを拡大
    return np.where(mask[:,:,None] == 255, frame, mosaic)  # マスクに基づいてモザイク部分を適用

def apply_blur_effect(frame, energy):
    """
    フレームにブラー効果を適用する関数

    Args:
        frame (numpy.ndarray): 処理対象のフレーム
        energy (float): 音声のエネルギー値 (0.0 - 1.0)

    Returns:
        numpy.ndarray: ブラー効果が適用されたフレーム
    """
    blur_amount = int(energy * 4)  # エネルギー値に基づいてブラーの強さを調整
    return cv2.GaussianBlur(frame, (blur_amount * 2 + 1, blur_amount * 2 + 1), 0)  # ガウシアンブラーを適用

def apply_bounce_effect(frame, current_onset, max_onset):
    """
    フレームにバウンス効果を適用する関数

    Args:
        frame (numpy.ndarray): 処理対象のフレーム
        current_onset (float): 現在のオンセット強度
        max_onset (float): 最大オンセット強度

    Returns:
        numpy.ndarray: バウンス効果が適用されたフレーム
    """
    bounce_amount = int(20 * current_onset / max_onset)  # オンセット強度に基づいてバウンスの高さを調整
    if bounce_amount > 0:
        padded_frame = cv2.copyMakeBorder(frame, bounce_amount, bounce_amount, 0, 0, cv2.BORDER_CONSTANT, value=[0, 0, 0])  # フレームの上下に黒い領域を追加
        return padded_frame[bounce_amount:-bounce_amount, :]  # バウンスしたように見える部分を切り出し
    else:
        return frame

def apply_glitch_effect(frame, strength=10):
    """
    フレームにグリッチ効果を適用する関数

    Args:
        frame (numpy.ndarray): 処理対象のフレーム
        strength (int, optional): グリッチの強さ。デフォルトは10。

    Returns:
        numpy.ndarray: グリッチ効果が適用されたフレーム
    """
    height, width, _ = frame.shape
    glitch_frame = frame.copy()
    num_slices = np.random.randint(1, strength)  # グリッチの数をランダムに決定
    for _ in range(num_slices):
        slice_height = np.random.randint(1, height // strength)  # グリッチの高さ
        start_y = np.random.randint(0, height - slice_height)  # グリッチの開始位置
        start_x = np.random.randint(-strength, strength)  # グリッチの水平方向のずれ
        end_x = width + start_x
        if start_x > 0:
            if end_x > width:
                end_x = width
            glitch_frame[start_y:start_y + slice_height, start_x:end_x] = frame[start_y:start_y + slice_height, :end_x - start_x]  # グリッチ部分をコピー
        else:
            if -start_x > width:
                start_x = -width
            glitch_frame[start_y:start_y + slice_height, :end_x] = frame[start_y:start_y + slice_height, -start_x:]  # グリッチ部分をコピー
    return glitch_frame


def create_music_visualizer(image_path, audio_path, output_path):
    """
    音楽ビジュアライザーを作成する関数

    Args:
        image_path (str): 入力画像のパス
        audio_path (str): 入力音声ファイルのパス
        output_path (str): 出力ビデオファイルのパス
    """
    y, sr = librosa.load(audio_path)  # 音声ファイルを読み込み
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=32, fmax=8000)  # メルスペクトログラムを計算
    S_dB = librosa.power_to_db(S, ref=np.max)  # デシベルに変換
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)  # オンセット強度を計算
    img = cv2.imread(image_path)  # 画像を読み込み
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # 色空間を変換
    height, width, _ = img.shape
    particles = create_particles(100, width, height)  # パーティクルを生成
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # ビデオコーデックを設定
    video = cv2.VideoWriter('temp_output.mp4', fourcc, 30, (width, height))  # ビデオライターを作成
    chunk_size = sr // 30  # 音声チャンクのサイズを計算
    prev_bars = np.zeros(32)  # 前のフレームのバーの高さを保持する変数
    zoom_factor = 1.0  # ズーム倍率
    mosaic_mask = create_mosaic_mask(height, width)  # モザイクマスクを作成
    max_onset = np.max(onset_env)  # 最大オンセット強度

    for i in range(0, len(y), chunk_size):  # 音声データをチャンクごとに処理
        chunk = y[i:i+chunk_size]  # 現在の音声チャンク
        spec_frame = S_dB[:, i//chunk_size] if i//chunk_size < S_dB.shape[1] else S_dB[:, -1]  # 現在のメルスペクトログラムフレーム
        current_onset = onset_env[i//chunk_size] if i//chunk_size < len(onset_env) else onset_env[-1]  # 現在のオンセット強度
        energy = np.mean(np.abs(chunk)) * 10  # 音声のエネルギー値を計算

        frame = img.copy()  # フレームをコピー
        frame = apply_mosaic_effect(frame, mosaic_mask)  # モザイク効果を適用
        particles = update_particles(particles, width, height, energy)  # パーティクルを更新
        draw_particles(frame, particles, (255, 255, 255))  # パーティクルを描画

        bar_width = width // 32  # バーの幅を計算
        max_bar_height = height // 4  # バーの最大の高さを計算

        for j, h in enumerate(spec_frame):  # メルスペクトログラムの値に基づいてバーを描画
            target_height = int(np.interp(h, [S_dB.min(), S_dB.max()], [0, max_bar_height]))  # バーの高さを計算
            prev_bars[j] = prev_bars[j] * 0.7 + target_height * 0.3  # 前のフレームの高さからスムーズに変化させる
            bar_height = int(prev_bars[j])

            # 半透明の白いバーを描画
            bar_color = (255, 255, 255, 150)  # 半透明の白
            outline_color = (0, 0, 0, 0)  # 不透明の黒

            # 上部のバー
            overlay = frame.copy()  # フレームをコピー
            cv2.rectangle(overlay,
                          (j * bar_width, height // 2 - bar_height),
                          ((j + 1) * bar_width, height // 2),
                          bar_color,
                          -1)  # バーを描画
            cv2.addWeighted(overlay, 0.5, frame, 1 - 0.5, 0, frame)  # フレームに合成

            # 上部のバーの輪郭
            cv2.rectangle(frame,
                          (j * bar_width, height // 2 - bar_height),
                          ((j + 1) * bar_width, height // 2),
                          outline_color,
                          1)  # 輪郭を描画

            # 下部のバー
            overlay = frame.copy()  # フレームをコピー
            cv2.rectangle(overlay,
                          (j * bar_width, height // 2),
                          ((j + 1) * bar_width, height // 2 + bar_height),
                          bar_color,
                          -1)  # バーを描画
            cv2.addWeighted(overlay, 0.5, frame, 1 - 0.5, 0, frame)  # フレームに合成

            # 下部のバーの輪郭
            cv2.rectangle(frame,
                          (j * bar_width, height // 2),
                          ((j + 1) * bar_width, height // 2 + bar_height),
                          outline_color,
                          1)  # 輪郭を描画

        overlay = frame.copy()
        cv2.addWeighted(overlay, 0.5, frame, 0.5, 0, frame)  # フレームに合成
        frame = apply_blur_effect(frame, energy)  # ブラー効果を適用
        target_zoom = 1 + 0.05 * current_onset / max_onset  # オンセット強度に基づいてズーム倍率を計算
        zoom_factor = zoom_factor * 0.7 + target_zoom * 0.3  # 前のフレームのズーム倍率からスムーズに変化させる
        scaled_frame = cv2.resize(frame, None, fx=zoom_factor, fy=zoom_factor)  # フレームを拡大縮小
        start_y = (scaled_frame.shape[0] - height) // 2
        start_x = (scaled_frame.shape[1] - width) // 2
        frame = scaled_frame[start_y:start_y + height, start_x:start_x + width]  # ズームした部分を切り出し
        frame = apply_bounce_effect(frame, current_onset, max_onset)  # バウンス効果を適用
        if np.random.rand() < 0.1:  # 10%の確率でグリッチ効果を適用
            frame = apply_glitch_effect(frame)
        video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # フレームをビデオライターに書き込み

    video.release()  # ビデオライターを解放
    video_clip = mpy.VideoFileClip("temp_output.mp4")  # ビデオクリップを読み込み
    audio_clip = mpy.AudioFileClip(audio_path)  # 音声クリップを読み込み
    final_clip = video_clip.set_audio(audio_clip)  # ビデオクリップに音声クリップを設定
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")  # ビデオファイルとして保存
    import os
    os.remove("temp_output.mp4")  # 一時ファイルを削除

# --- 使用例 ---
image_path = 'image.png'  # 入力画像のパス
audio_path = 'audio.mp3'  # 入力音声ファイルのパス
output_path = 'output_video.mp4'  # 出力ビデオファイルのパス
create_music_visualizer(image_path, audio_path, output_path)  # ビジュアライザーを作成